In [5]:
%load_ext autoreload

In [10]:
%autoreload 2
import pandas as pd
import numpy as np
from data_pre_proc import read_data, filter_data, filter_by_earnigs

In [8]:
df = read_data('https://www.dropbox.com/s/3avq0b4fsbujtvp/pequiv_long.dta?dl=1')

In [3]:
data_dict = {
        # Identifiers
        'year' : "Year",
        'x11101LL' : 'id',
        'x11102' : 'Household id',

        # Demographic variables
        # "d11101" : "Age",
        # "d11102LL" : "Sex",
        # "h11112" : "Spouse in HH",
        # "d11108" : "Education",
        # "d11112LL" : "Race",
        # "d11106" : "Household Size",

        # Employment variables
        "e11101" : "Annual Work Hours",
        "e11102" : "Employment Status",

        # Income variables
        "i11113" : "After-tax Income"
        }

In [68]:
df_1 = filter_data(df, 1978, 1997, data_dict, {'Employment Status' : ['Employed       1']})

df_1

,Year,id,Household id,Annual Work Hours,Employment Status,After-tax Income
19,1979.0,1003.0,3273.0,2256.0,Employed 1,23500.63
21,1978.0,1003.0,2476.0,3322.0,Employed 1,21416.34
26,1979.0,1004.0,3273.0,2028.0,Employed 1,23500.63
35,1978.0,1004.0,2476.0,1935.0,Employed 1,21416.34
73,1985.0,2171.0,902.0,780.0,Employed 1,3828.00
...,...,...,...,...,...,...
440399,1997.0,3435003.0,10211.0,480.0,Employed 1,18168.00
440416,1997.0,3435005.0,10211.0,2080.0,Employed 1,18168.00
440473,1997.0,3436003.0,10237.0,360.0,Employed 1,4157.04
440590,1997.0,3438003.0,10268.0,240.0,Employed 1,3000.00


In [13]:
# Filter outliers

In [77]:
# Full time employment is 130 hours per month
full_time_hours = 130*12# = 1560
employed_id = {}

grouped_by_year = df_1.groupby("Year")
for g, d in grouped_by_year:
    subsample = d[d["Annual Work Hours"] > full_time_hours]
    employed_id[g] = subsample.id.values

years = list(employed_id.keys())

income_diff_df = pd.DataFrame({"Year": [], "id": [], "delta_income": []})

for i in range(len(years) - 1):
    employed_today = employed_id[years[i]]
    employed_next = employed_id[years[i+1]]
    consecutive_employed = np.intersect1d(employed_today, employed_next)
    data_today = grouped_by_year.get_group(years[i]).set_index("id").loc[consecutive_employed]
    data_next = grouped_by_year.get_group(years[i+1]).set_index("id").loc[consecutive_employed]
    data_next["delta_income"] = data_next["After-tax Income"] - data_today["After-tax Income"]
    income_diff_df = pd.concat([income_diff_df, data_next[["Year", "delta_income", "After-tax Income"]].reset_index()])
    
income_diff_df

,Year,id,delta_income,After-tax Income
0,1979.0,1003.0,2084.29,23500.63
1,1979.0,1004.0,2084.29,23500.63
2,1979.0,4003.0,2868.52,15242.32
3,1979.0,4170.0,1168.03,10531.81
4,1979.0,4172.0,583.27,14482.30
...,...,...,...,...
3652,1997.0,2924170.0,-53339.27,73080.90
3653,1997.0,2927003.0,3455.91,51062.83
3654,1997.0,2927030.0,2283.55,25676.70
3655,1997.0,2927031.0,31425.54,50209.63


In [78]:
income_diff_df.groupby("Year").agg({"delta_income": ["mean", "std"]})

delta_income              
               mean           std
Year                             
1979.0  1615.755985   5471.342818
1980.0  1753.693636   6593.952912
1981.0  1960.638747  11108.426299
1982.0  1445.763832  11301.110959
1983.0  1923.408473   8851.846959
1984.0  2466.788278   7965.118709
1985.0  2289.228349  13625.741843
1986.0  1660.206533  12264.434445
1987.0  1586.565589  10230.739587
1988.0  4050.434676  14965.083258
1989.0  3232.975977  13813.335223
1990.0  1908.295498  14926.476568
1991.0  1660.738727  16718.266975
1992.0  2189.461415  16042.843460
1993.0  3221.423656  22579.388328
1994.0   561.822510  30262.960000
1995.0  1504.378769  30516.767855
1996.0  2744.781468  24194.174092
1997.0  5233.197553  25770.535021

In [79]:
income_diff_df.groupby("Year").get_group(1997)

,Year,id,delta_income,After-tax Income
0,1997.0,4003.0,15362.58,46730.48
1,1997.0,4004.0,-14736.32,74754.47
2,1997.0,4006.0,44455.04,94567.19
3,1997.0,4186.0,-12620.74,69957.45
4,1997.0,4188.0,44455.04,94567.19
...,...,...,...,...
3652,1997.0,2924170.0,-53339.27,73080.90
3653,1997.0,2927003.0,3455.91,51062.83
3654,1997.0,2927030.0,2283.55,25676.70
3655,1997.0,2927031.0,31425.54,50209.63


In [80]:
df_1[df_1["id"] == 4006.0].sort_values("Year")

,Year,id,Household id,Annual Work Hours,Employment Status,After-tax Income
255,1978.0,4006.0,5849.0,175.00,Employed 1,9363.78
277,1985.0,4006.0,1344.0,176.00,Employed 1,15492.00
261,1986.0,4006.0,1310.0,380.00,Employed 1,18749.10
268,1987.0,4006.0,2332.0,390.00,Employed 1,20342.43
266,1988.0,4006.0,1216.0,537.00,Employed 1,19570.00
269,1989.0,4006.0,3235.0,615.00,Employed 1,23457.65
271,1990.0,4006.0,1757.0,409.00,Employed 1,22438.10
245,1992.0,4006.0,6974.0,327.00,Employed 1,26864.35
249,1993.0,4006.0,8042.0,2040.00,Employed 1,9879.00
256,1994.0,4006.0,11739.0,1733.36,Employed 1,9800.15


In [81]:
income_diff_df[income_diff_df["id"] == 4006.0].sort_values("Year")

,Year,id,delta_income,After-tax Income
1,1994.0,4006.0,-78.85,9800.15
1,1995.0,4006.0,1514.74,11314.89
2,1996.0,4006.0,38797.26,50112.15
2,1997.0,4006.0,44455.04,94567.19


In [83]:
income_diff_df.rename(columns={"After-tax Income": "income"}, inplace=True) #TODO: solve this when selecting the right column
income_diff_df.to_csv("/home/mitch34/Work/UW Courses/04 - 2022 Fall/Econ 810 Advanded Macroeconomic Theory/Part 1/PS 2/data/income_diff.csv", index=False)